In [1]:
# import our libraries
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
import pprint
import pandas
import sys
import os
import timeit
import numpy as np
import logging
sys.setrecursionlimit(100000)
%load_ext line_profiler
import urllib.request


In [2]:
# Ticker to CIK file obtained from:
# https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm##targetText=The%20CIK%20is%20the%20unique,filer%3B%20they%20are%20not%20recycled.

cik=pandas.read_csv('ticker_cik.txt', sep='\t', lineterminator='\n')
SP500= pandas.read_csv('SP500.csv')
SP500['co_tic']=SP500['co_tic'].str.lower()

cik_SP500=pandas.merge(SP500, cik, on='co_tic')

In [8]:

start = timeit.default_timer()

# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"
for k in range(43,100):  # On teste avec les 50 premières compagnies du SP500
    cik=cik_SP500.CIK[k]
    ticker=cik_SP500.co_tic[k]
    con_name=cik_SP500.co_conm[k]
    os.mkdir('{0}'.format(ticker))  # We want to save the reports in a directory named after the company ticker
    save_path = ('{0}/'.format(ticker))

    # define our parameters dictionary
    param_dict = {'action':'getcompany',
                  'CIK':cik, #Can also use: 'Company': 'Microsoft',
                  #'company':'Microsoft',
                  'type':'10-K',
                  'dateb':'20190101',
                  'owner':'exclude',
                  'start':'',
                  'output':'atom',
                  'count':'100'}

    # request the url, and then parse the response.
    response = requests.get(url = endpoint, params = param_dict)  # This is the url of the search results, not the annual report itself
    soup = BeautifulSoup(response.content, 'lxml')

    # Let the user know it was successful.
    print('Request Successful')
    print(response.url)

    # find all the entry tags
    entries = soup.find_all('entry')

    # initalize our list for storage
    master_list_xml = []
    accession_num_saved=[]
    # loop through each found entry, remember this is only the first two
    #for entry in entries[0:2]:
    for entry in entries:

        # grab the accession number so we can create a key value
        accession_num = entry.find('accession-nunber').text
        accession_num_saved.append(accession_num)

        # create a new dictionary
        entry_dict = {}
        entry_dict[accession_num] = {}


        # store the file info
        entry_dict[accession_num]['file_info'] = {}

        entry_dict[accession_num]['file_info']['filing_date'] =  entry.find('filing-date').text
        entry_dict[accession_num]['file_info']['filing_href'] = entry.find('filing-href').text


        entry_dict[accession_num]['request_info'] = {}
        entry_dict[accession_num]['request_info']['link'] =  entry.find('link')['href']
        entry_dict[accession_num]['request_info']['title'] =  entry.find('title').text
        entry_dict[accession_num]['request_info']['last_updated'] =  entry.find('updated').text

        # store in the master list
        master_list_xml.append(entry_dict)


        # Loop to save the files on a local disk
        for z in range(0,len(master_list_xml)):
        
            # We want only the annual report from 2000-01-01
            if master_list_xml[z][accession_num_saved[z]]['file_info']['filing_date']>='2000-01-01':
                url_2=master_list_xml[z][accession_num_saved[z]]['request_info']['link']
                response2 = requests.get(url=url_2,stream=True)
                soup2 = BeautifulSoup(response2.content, 'lxml')
                target=soup2.findAll("table", {"class": "tableFile"})
                target2=target[0].findAll('tr')
                target3=target2[1].findAll('a',href=True)
                html=target3[0]['href']
                prefix='https://www.sec.gov/'
                finalhtml=prefix+html
                # grab the response
                response = requests.get(finalhtml, stream=True)
                content=response.content
                file_name=accession_num_saved[z]

                #if soup.find('PDF'):        ## remove PDF from file
                   # soup.PDF.decompose()
                name_of_file = ('%s.html' % file_name)   # The file will be named with the accession number of the file
                completeName = os.path.join(save_path, name_of_file)         
                open(completeName, "wb").write(content)

stop = timeit.default_timer()
print('Time: ', stop - start)  



Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=80424&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=788784&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1047122&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1024478&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=827052&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=92122&type=10-K&dateb=20190101&owner=exclude&start=&output=atom&count=100
Request Successful
https://www.sec.gov/cgi-bin/browse-edga

99
